# `ipython`
> Set of utility functions to be used in Jupyter and Jupyter Lab notebooks.


In [ ]:
#|default_exp ipython

In [ ]:
#| export
from __future__ import annotations
from fastcore.test import test_fail
from functools import wraps
from IPython.core.getipython import get_ipython
from IPython.display import display, Markdown, display_markdown
from pathlib import Path
from typing import Any, Callable, Optional
from ecutilities.core import validate_path, is_type, IsLocalMachine

import configparser
import numpy as np
import os
import pandas as pd
import subprocess
import warnings

In [ ]:
#| hide
from nbdev import show_doc, nbdev_export

# System and CLI

In [ ]:
#| export
def run_cli(cmd:str = 'ls -l'   # command to execute in the cli
           ):
    """Runs a cli command from jupyter notebook and print the shell output message
    
    Uses subprocess.run with passed command to run the cli command"""
    p = subprocess.run(cmd, stdout=subprocess.PIPE, shell=True)
    print(str(p.stdout, 'utf-8'))

In [ ]:
run_cli('pwd')

/home/vtec/projects/ec-packages/ecutilities/nbs-dev



# Notebook setup

In [ ]:
#| export
def nb_setup(autoreload:bool = True,   # True to set autoreload in this notebook
             paths:list(Path) = None   # Paths to add to the path environment variable
            ):
    """Use in first cell of notebook to set autoreload, and paths"""
#   Add paths. Default is 'src' if it exists
    if paths is None:
        p = Path('../src').resolve().absolute()
        if p.is_dir():
            paths = [str(p)]
        else:
            paths=[]
    if paths:
        for p in paths:
            sys.path.insert(1, str(p))
        print(f"Added following paths: {','.join(paths)}")

#   Setup auto reload
    if autoreload:
        ipshell = get_ipython()
        ipshell.run_line_magic('load_ext',  'autoreload')
        ipshell.run_line_magic('autoreload', '2')
        print('Set autoreload mode')

In [ ]:
show_doc(nb_setup)

---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/ipython.py#L34){target="_blank" style="float:right; font-size:smaller"}

### nb_setup



Use in first cell of notebook to set autoreload, and paths

In [ ]:
nb_setup()

Set autoreload mode


By default, `ipython.nb_setup()` 
- loads and set `autoreload`
- adds a path to a directory named `src` when it exists at the same level as where the notebook directory is located. It no such `src` directory exists, no path is added

`ipython.nb_setup` assumes the following file structure:

```
    project_directory
          | --- notebooks
          |        | --- current_nb.ipynb
          |        | --- ...
          |
          |--- src
          |     | --- scripts_to_import.py
          |     | --- ...
          |
          |--- data
          |     |
          |     | ...
```

For other file structure, specify paths as a `list` of `Path`

In [ ]:
#| export
def cloud_install_project_code(
    package_name:str # project package name, e.g. metagentools or git+https://github.com/repo.git@main
):
    """When nb is running in the cloud, pip install the project code package"""
    
    # test whether it runs on colab
    try:
        from google.colab import drive
        RUN_LOCALLY = False
        print('The notebook is running on colab')

    except ModuleNotFoundError:
        # not running on colab, testing is it runs on on a local machine
        RUN_LOCALLY = IsLocalMachine().is_local
        
        if RUN_LOCALLY:
            print('The notebook is running locally, will not automatically install project code')
        else:
            print('The notebook is running on a cloud VM or the machine was not registered as local')

    if not RUN_LOCALLY:
        print(f'Installing project code {package_name}')
        cmd = f"pip install -U {package_name}"
        run_cli(cmd)
        print((f"{package_name} is installed."))
        
    return RUN_LOCALLY

When using colab or another cloud VM, project code must be installed every time from the Python Package Index (PyPI) or its GitHub repo.

When running locally, the project code should be pre-installed as part of the environment

In [ ]:
cloud_install_project_code(package_name='metagentools');

The notebook is running locally, will not automatically install project code


# Improve output cell formats

In [ ]:
#| export
def display_mds(
    *strings:str|tuple[str] # any number of strings with text in markdown format
):
    """Display one or several strings formatted in markdown format"""
    for string in strings:
        display_markdown(Markdown(data=string))

In [ ]:
show_doc(display_mds)

---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/ipython.py#L87){target="_blank" style="float:right; font-size:smaller"}

### display_mds

>      display_mds (*strings:str|tuple[str])

Display one or several strings formatted in markdown format

In [ ]:
display_mds('**bold** and _italic_')

**bold** and _italic_

In [ ]:
display_mds('**bold** and _italic_',
            '- bullet',
            '- bullet',
            '> Note: this is a note'
)

**bold** and _italic_

- bullet

- bullet

> Note: this is a note

In [ ]:
#| export
def display_dfs(*dfs:pd.DataFrame       # any number of Pandas DataFrames
               ):
    """Display one or several `pd.DataFrame` in a single cell output"""
    for df in dfs:
        display(df)

In [ ]:
show_doc(display_dfs)

---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/ipython.py#L95){target="_blank" style="float:right; font-size:smaller"}

### display_dfs

>      display_dfs (*dfs:pandas.core.frame.DataFrame)

Display one or several `pd.DataFrame` in a single cell output

In [ ]:
df1 = pd.DataFrame(data=np.random.normal(size=(10,5)))
df2 = pd.DataFrame(data=np.random.normal(size=(20,10)))

display_dfs(df1.head(3), df2.head(3))

,0,1,2,3,4
0,0.255490,-1.171769,1.415709,0.229384,0.329095
1,-0.299455,2.034436,-0.373203,0.432097,1.224080
2,0.346878,0.371325,1.022736,-1.634051,0.917244


,0,1,2,3,4,5,6,7,8,9
0,-0.561503,-0.764531,-0.976769,0.792954,-0.066988,0.829037,0.531817,-0.836533,1.162198,0.197128
1,1.133557,0.558838,0.321028,-0.031272,1.215234,-0.559245,-0.347999,-0.025495,-1.558692,2.212615
2,-0.325063,-2.076578,0.850863,-0.456210,-1.582164,-1.788974,0.915132,2.211256,-0.623770,-0.339751


In [ ]:
#| export
class pandas_nrows_ncols:
    """Context manager set max number of rows and cols to apply to any output within the context"""
    def __init__(
        self, 
        nrows:int|None=None, # max number of rows to show; show all rows if `None`
        ncols:int|None=None, # max number of columns to show; show all columns if `None`
    ):
        self.nrows = nrows
        self.ncols = ncols
    
    def __enter__(self):
        self.max_rows = pd.options.display.max_rows
        self.max_cols = pd.options.display.max_columns
        pd.options.display.max_rows = self.nrows
        pd.options.display.max_columns = self.ncols
        return self.max_rows, self.max_cols

    def __exit__(self, exc_type, exc_value, exc_tb):
        pd.options.display.max_rows = self.max_rows
        pd.options.display.max_columns = self.max_cols

In [ ]:
show_doc(pandas_nrows_ncols)

---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/ipython.py#L102){target="_blank" style="float:right; font-size:smaller"}

### pandas_nrows_ncols

>      pandas_nrows_ncols (nrows:int|None=None, ncols:int|None=None)

Context manager set max number of rows and cols to apply to any output within the context

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| nrows | int \| None | None | max number of rows to show; show all rows if `None` |
| ncols | int \| None | None | max number of columns to show; show all columns if `None` |

With no context manager, the pandas object are displayed with a maximum of 60 rows and 20 columns.

In [ ]:
df = pd.DataFrame(np.random.randint(low=0, high=100, size=(3,50)))
display(df)

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,16,10,65,8,28,17,36,70,7,79,...,72,61,2,8,43,0,95,87,0,2
1,7,96,95,16,26,92,27,58,58,95,...,89,12,12,66,99,37,4,45,47,22
2,76,11,79,80,50,98,82,13,0,37,...,67,37,87,42,99,71,60,19,46,76


Using the context manager, all rows and columns will be displayed

In [ ]:
with pandas_nrows_ncols():
    display(df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,16,10,65,8,28,17,36,70,7,79,58,98,98,2,18,87,9,43,62,16,20,37,81,34,20,75,6,67,68,36,63,64,57,60,18,4,50,24,11,91,72,61,2,8,43,0,95,87,0,2
1,7,96,95,16,26,92,27,58,58,95,34,18,38,68,41,27,64,46,98,44,92,14,64,17,1,14,30,29,63,79,34,36,92,30,81,37,97,80,64,3,89,12,12,66,99,37,4,45,47,22
2,76,11,79,80,50,98,82,13,0,37,98,7,30,78,86,27,12,11,40,62,41,69,98,55,21,74,59,25,25,87,32,58,19,17,30,49,2,58,90,44,67,37,87,42,99,71,60,19,46,76


It is also possible to specifically define the number of rows and columns to display

In [ ]:
with pandas_nrows_ncols(nrows=2, ncols=4):
    display(df)

,0,1,...,48,49
0,16,10,...,0,2
...,...,...,...,...,...
2,76,11,...,46,76


> **Technical background**:
> 
> the context manager uses pandas's [`options API`](https://pandas.pydata.org/docs/user_guide/options.html)

In [ ]:
pd.options.display.max_rows, pd.options.display.max_columns

(60, 20)

In [ ]:
pd.get_option('display.max_rows'), pd.get_option('display.max_columns')

(60, 20)

In [ ]:
pd.describe_option('display.max_rows')

display.max_rows : int
    If max_rows is exceeded, switch to truncate view. Depending on
    `large_repr`, objects are either centrally truncated or printed as
    a summary view. 'None' value means unlimited.

    In case python/IPython is running in a terminal and `large_repr`
    equals 'truncate' this can be set to 0 and pandas will auto-detect
    the height of the terminal and print a truncated object which fits
    the screen height. The IPython notebook, IPython qtconsole, or
    IDLE do not run in a terminal and hence it is not possible to do
    correct auto-detection.
    [default: 60] [currently: 60]


In [ ]:
pd.options.display.max_rows = 10
pd.reset_option('display.max_rows')
pd.options.display.max_rows

60

In [ ]:
#| export
def df_all_cols_and_rows(
    f:Callable,   # function to apply the decorator ti
)-> Callable:     # decorated function
    """decorator function forcing all rows and columns of `DataFrames` to be displayed in the wrapped function"""
    
    msg = 'This decorator is deprecated. Will be removed soon. Use context manager `pandas_nrows_ncols` instead.'
    warnings.warn(msg, category=DeprecationWarning)
    
    @wraps(f)
    def wrapper(*args, **kwargs):
        max_rows = pd.options.display.max_rows
        max_cols = pd.options.display.max_columns
        pd.options.display.max_rows = None
        pd.options.display.max_columns = None
        f(*args, **kwargs)
        pd.options.display.max_rows = max_rows
        pd.options.display.max_columns = max_cols
    
    return wrapper

In [ ]:
show_doc(df_all_cols_and_rows)

---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/ipython.py#L124){target="_blank" style="float:right; font-size:smaller"}

### df_all_cols_and_rows

>      df_all_cols_and_rows (f:Callable)

decorator function forcing all rows and columns of `DataFrames` to be displayed in the wrapped function

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| f | Callable | function to apply the decorator ti |
| **Returns** | **Callable** | **decorated function** |

Usage of the decorator

In [ ]:
#| hide
@df_all_cols_and_rows
def show_df(df):
    display(df)

df = pd.DataFrame(np.random.randint(low=0, high=100, size=(3,50)))
show_df(df)

/tmp/ipykernel_1801/1674591720.py:8: DeprecationWarning: This decorator is deprecated. Will be removed soon. Use context manager `pandas_nrows_ncols` instead.
  warnings.warn(msg, category=DeprecationWarning)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,3,48,25,53,22,86,12,81,48,94,19,10,93,96,22,0,17,76,11,96,83,82,5,8,94,80,12,54,22,88,33,74,91,98,94,92,22,1,88,45,52,15,80,63,66,25,91,6,53,36
1,97,98,83,19,4,53,53,65,85,16,1,81,90,74,88,72,42,0,9,49,54,11,5,73,97,36,39,24,27,1,87,61,29,11,74,35,31,39,32,64,95,22,88,95,33,0,81,86,4,84
2,10,16,63,92,89,71,93,85,37,16,61,68,66,33,37,55,72,22,24,4,86,43,16,99,91,27,74,59,11,52,79,43,48,12,94,76,15,91,58,6,60,73,49,68,0,11,83,72,52,64


In [ ]:
#| export
def display_full_df(
    df:pd.DataFrame|pd.Series,  # `DataFrame` or `Series` to display
):
    """Display a pandas `DataFrame` or `Series` showing all rows and columns"""
    if is_type(df, pd.DataFrame, raise_error=False) or is_type(df, pd.Series, raise_error=False):
        with pandas_nrows_ncols():
            display(df)
    else:
        raise TypeError(f"df must me a pandas `DataFrame` or `Series`, not a {type(df)}")

In [ ]:
show_doc(display_full_df)

---

[source](https://github.com/vtecftwy/ecutils/blob/master/ecutilities/ipython.py#L145){target="_blank" style="float:right; font-size:smaller"}

### display_full_df

>      display_full_df
>                       (df:pandas.core.frame.DataFrame|pandas.core.series.Serie
>                       s)

Display a pandas `DataFrame` or `Series` showing all rows and columns

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| df | pd.DataFrame \| pd.Series | `DataFrame` or `Series` to display |

In [ ]:
df = pd.DataFrame(np.random.randint(low=0, high=100, size=(3,50)))
df

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,93,59,82,45,74,18,84,75,19,70,...,43,3,2,87,3,61,94,5,39,11
1,35,48,93,76,6,2,36,80,49,39,...,68,1,29,7,30,40,83,65,38,35
2,87,5,62,98,96,53,27,85,93,73,...,98,75,68,44,0,40,55,87,46,56


In [ ]:
display_full_df(df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,93,59,82,45,74,18,84,75,19,70,66,68,67,97,95,7,50,71,66,77,69,86,18,93,24,70,99,53,37,65,65,95,41,93,54,70,87,77,67,97,43,3,2,87,3,61,94,5,39,11
1,35,48,93,76,6,2,36,80,49,39,17,20,70,2,14,25,67,23,60,46,15,85,71,40,61,58,50,50,30,80,50,1,51,67,42,8,71,41,47,28,68,1,29,7,30,40,83,65,38,35
2,87,5,62,98,96,53,27,85,93,73,72,83,95,85,63,31,79,69,55,64,67,6,79,70,42,46,76,82,7,52,18,80,35,20,60,87,14,56,27,5,98,75,68,44,0,40,55,87,46,56


In [ ]:
#| hide
display_full_df(df.loc[0, :].T)

0     93
1     59
2     82
3     45
4     74
5     18
6     84
7     75
8     19
9     70
10    66
11    68
12    67
13    97
14    95
15     7
16    50
17    71
18    66
19    77
20    69
21    86
22    18
23    93
24    24
25    70
26    99
27    53
28    37
29    65
30    65
31    95
32    41
33    93
34    54
35    70
36    87
37    77
38    67
39    97
40    43
41     3
42     2
43    87
44     3
45    61
46    94
47     5
48    39
49    11
Name: 0, dtype: int64

In [ ]:
msg = 'should raise a TypeError'
contains = 'df must me a pandas `DataFrame` or `Series`'

test_fail(display_full_df, args=['a string'], msg=msg, contains=contains)

In [ ]:
#| hide
nbdev_export()